In [4]:
import json
import os
import pandas as pd
import numpy as np
from typing import List, Dict
os.chdir("/Users/t.dopierre/Projects/nlp-analyser/server")

# Lara Data

In [8]:
def save_txt(texts: List[str], path:str):
    with open(path, "w") as file:
        for line in texts:
            file.write(line+"\n")

def save_jsonl(docs: List[Dict], path: str):
    with open(path, "w") as file:
            for doc in docs:
                file.write(json.dumps(doc, ensure_ascii=False)+"\n")

In [4]:
for sheet_name in ("Verbatims 1d (fr)", "Verbatims 3b (fr)"):
    df = pd.read_excel('data/data-care/raw/Verbatims FR Women Safety V2 POC Erdil + méta cohorte.xlsx', sheet_name=sheet_name)
    texts = df[df.columns[0]].astype(str).tolist()
    save_txt(texts, f'data/data-care/parsed/Lara {sheet_name}.txt')

# Care data

In [5]:
def care_df_to_datasets(df):
    dataset_categories, dataset_sentiment = list(), list()
    for ix, row in df.iterrows():
        text = str(row.Message).replace("\xa0"," ")
        categories = [
            cat.strip()
            for col in ("Concept - positive tonality", "Concept - negative tonality", "Concept - neutral tonality")
            for cat in str(row[col]).split('\n') if type(row[col])==str
        ]

        dataset_categories.append({
            "text": text,
            "labels": categories
        })

        if row["Tonalité"] in ("Positive","Neutre","Négative"):
            dataset_sentiment.append({
                "text": text,
                "label": row["Tonalité"]
            })
    return dataset_categories, dataset_sentiment

In [6]:
file_path = "data/data-care/raw/enquetes_all_messages_01-11-2021_30-11-2021.xlsx"
dataset_categories, dataset_sentiment = care_df_to_datasets(pd.read_excel(file_path, sheet_name="Messages"))
save_jsonl(dataset_categories, "data/data-care/parsed/enquetes-nov-categories.jsonl")
save_jsonl(dataset_sentiment, "data/data-care/parsed/enquetes-nov-sentiment.jsonl")

In [7]:
file_path = "data/data-care/raw/requetes_all_messages_06-12-2021_12-12-2021.xlsx"
dataset_categories, dataset_sentiment = care_df_to_datasets(pd.read_excel(file_path, sheet_name="Messages"))
save_jsonl(dataset_categories, "data/data-care/parsed/requetes-dec-categories.jsonl")
save_jsonl(dataset_sentiment, "data/data-care/parsed/requetes-dec-sentiment.jsonl")

In [6]:
path = "../data/data-care/parsed/requetes-dec-sentiment.jsonl"

In [16]:
from tqdm.notebook import tqdm

In [7]:
from src.ml.classification.models import TransformerClassifier
from src.ml.data import DataSample, Dataset

In [8]:
with open(path, "r") as file:
    data = [DataSample(**json.loads(line.strip())) for line in file]
dataset = Dataset(data)

In [9]:
t = TransformerClassifier(embedder_class="transformer", embedder_config={"model_name_or_path":"bert-base-multilingual-cased"})

DEBUG:TransformerClassifier:Embedding model class: transformer / config: {'model_name_or_path': 'bert-base-multilingual-cased'}
Some weights of the model checkpoint at /Users/t.dopierre/.cache/torch/sentence_transformers/bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

In [10]:
t.embedder.fit(dataset)

In [14]:
x = t.embedder.predict(["hey"]*100)

In [18]:
embeddings = t.embedder.predict([sample.text for sample in tqdm(dataset[:100])])

  0%|          | 0/100 [00:00<?, ?it/s]

In [47]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

In [48]:
assert (sample.labels for sample in dataset)
assert (sample.embedding for sample in dataset)
self = t
self.unique_labels = sorted(set([label for sample in dataset for label in sample.labels]))
self.label_to_ix = {v: k for k, v in enumerate(self.unique_labels)}
self.ix_to_label = {k: v for k, v in enumerate(self.unique_labels)}
y = np.zeros(shape=(len(dataset), len(self.unique_labels)))
for sample_ix, sample in enumerate(dataset):
    for label in sample.labels:
        y[sample_ix, self.label_to_ix[label]] = 1

model = MultiOutputClassifier(estimator=LogisticRegression())

In [55]:
model.fit(embeddings, y)

/home/tdopierre/Projects/nlp-analyser/server/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tdopierre/Projects/nlp-analyser/server/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-

MultiOutputClassifier(estimator=LogisticRegression())

In [17]:
from copy import deepcopy
dataset_copy = deepcopy(dataset)
assert len(embeddings) == len(dataset_copy)
for sample, embedding in zip(dataset_copy, embeddings):
    sample.embedding = embedding

In [ ]:
t.fit(dataset)

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [90]:
from sklearn.datasets import make_blobs

In [91]:
X, y = make_blobs()

In [106]:
y = (np.random.randn(X.shape[0], 5) > 0.5).astype(int)

In [ ]:
y = 

In [114]:
rf = RandomForestClassifier()

In [111]:
lr = LogisticRegression(multi_class="ovr")

In [129]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

X, y = make_multilabel_classification(n_classes=3, random_state=0)
clf = MultiOutputClassifier(estimator=LogisticRegression()).fit(X, y)
clf.predict(X[-2:])

array([[1, 1, 1],
       [1, 0, 1]])

In [134]:
clf.predict_proba(X)

[array([[9.43300313e-01, 5.66996865e-02],
        [8.82995057e-01, 1.17004943e-01],
        [6.16805013e-01, 3.83194987e-01],
        [7.76668363e-01, 2.23331637e-01],
        [9.66934855e-01, 3.30651447e-02],
        [5.04741580e-01, 4.95258420e-01],
        [9.11686786e-01, 8.83132137e-02],
        [1.26783030e-03, 9.98732170e-01],
        [9.24669395e-01, 7.53306054e-02],
        [9.78157945e-01, 2.18420554e-02],
        [5.28145217e-01, 4.71854783e-01],
        [6.93068115e-01, 3.06931885e-01],
        [2.73046314e-01, 7.26953686e-01],
        [3.78035398e-04, 9.99621965e-01],
        [2.41595006e-01, 7.58404994e-01],
        [3.55717754e-03, 9.96442822e-01],
        [9.04571974e-01, 9.54280257e-02],
        [7.46187843e-01, 2.53812157e-01],
        [9.49409412e-01, 5.05905876e-02],
        [2.39201911e-02, 9.76079809e-01],
        [9.83734785e-01, 1.62652146e-02],
        [9.98975503e-01, 1.02449681e-03],
        [2.71030026e-01, 7.28969974e-01],
        [4.36541742e-01, 5.6345825

In [130]:
from sklearn.metrics import f1_score

In [133]:
f1_score(y, clf.predict(X), average="micro")

0.88

In [128]:
clf.predict(X)

array([[1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
    

In [127]:
clf.predict(X)==y

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [125]:
rf.predict(X) == y

ValueError: Number of features of the model must match the input. Model n_features is 2 and input n_features is 20 